Watch DM progress

In [1]:
import os
import time
import numpy as np
import xarray as xr
result_dir = '/usr/scratch4/sima9999/work/modal_uq/uq_modal_beam/'
path = os.path.join(result_dir, 'samples')

In [4]:
import matplotlib
matplotlib.use('qtagg')
import matplotlib.pyplot as plt
plt.figure()

In [5]:
plt.show()

In [4]:

in_ds = xr.load_dataset(path + '/uq_modal_beam.nc')
# out_ds = xr.load_dataset(path + '/uq_modal_beam_out.nc')
# np.isnan(out_ds._runtimes.data)[0][0]

In [ ]:
for i in range(3999242,7599653): # 3938021 is the first invalid # 3625888 is broken, is not broken?
    if not i%250:
        print('.', end='', flush=True)
    jid = in_ds.ids[i].item()
    res_dir = os.path.join(path,jid,f'{jid}_mechanical.npz')
    arr = np.load(res_dir, allow_pickle=True)
    assert arr['self.struct_parms'].item()['dy_tmd'] == in_ds['dD'][i].item()

In [5]:
display(i)

3999242

In [38]:
jid = in_ds.ids[i].item()
res_dir = os.path.join(path,jid,f'{jid}_mechanical.npz')
print(time.ctime(os.stat(res_dir).st_mtime))
print(time.ctime(os.stat(res_dir).st_ctime))

Mon Nov  7 09:21:49 2022
Mon Nov  7 09:21:49 2022


- 9,999,693 = 729 * 13,717 # total samples
- 3,930,727 / 729 = 5,391.9 # last datamanager sample with runtime not nan -> start_ale=5390 is the last good sample when plotting in estimate_imp
- 6,581,412 = 729 * 9,028 # last datamanager sample with results not nan
- 3,938,021 / 729 = 5,401.9 # last sample, that progress watcher marks valid, probably the last 7294 samples were not saved into db -> it is bad when plotting in estimate_imp

In [80]:
result_dir = '/usr/scratch4/sima9999/work/modal_uq/uq_modal_beam/'
for ret_name in ['damp_freqs','zetas','frf']:
    
    if ret_name == 'frf':
        # continue
        inds = range(1025*3)
    else:
        continue
        inds = range(14)
    all_times = []
    all_dirs = []
    all_progr = []
    for ind in inds:
        if ret_name == 'frf':
            ret_ind = {'frequencies':ind//3, 'space':ind%3}
            if  ind%3==0: continue
        else:
            ret_ind = {'modes':ind}
            
        ret_dir = f'{ret_name}-{".".join(str(e) for e in ret_ind.values())}'
        fname = os.path.join(result_dir, 'estimations', f'{ret_dir}/polyuq_imp.npz')
        # print(fname)
        if os.path.exists(fname):
            all_times.append(os.stat(fname).st_mtime)
            all_dirs.append(ret_dir)
            try:
                intp_err = np.load(fname)['self.intp_errors']
            except:
                all_progr.append(0)
                continue
            nans = np.isnan(intp_err)
            if ind%3==2:
                # break
                nans[:5392]=0            
            nan_ind = np.where(nans)[0]
            if len(nan_ind):
                all_progr.append(np.min(nan_ind))
            else:
                all_progr.append(len(intp_err))
        else:
            all_progr.append(0)
            all_dirs.append(ret_dir)
            all_times.append(0)
    
    display(f'finished: {np.sum(np.array(all_progr)==13717)}, not finished {np.sum(np.array(all_progr)!=13717)}')
    # for ret_time,ret_dir, progr in sorted(zip(all_times, all_dirs, all_progr)):
    for ret_time,ret_dir, progr in zip(all_times, all_dirs, all_progr):
        # if progr == 5390: continue
        # if 1675933971.0 <= ret_time <= 1675934235.0 and progr==0: continue
        print(f'{ret_dir} @ {progr} on {time.ctime(ret_time)} {ret_time}')

'finished: 1273, not finished 777'

frf-0.1 @ 13717 on Tue Feb 28 18:12:24 2023 1677604344.0
frf-0.2 @ 13717 on Tue Feb 28 16:25:13 2023 1677597913.0
frf-1.1 @ 13717 on Tue Feb 28 18:19:00 2023 1677604740.0
frf-1.2 @ 13717 on Tue Feb 28 15:57:58 2023 1677596278.0
frf-2.1 @ 13717 on Tue Feb 28 18:05:17 2023 1677603917.0
frf-2.2 @ 13717 on Tue Feb 28 15:56:21 2023 1677596181.0
frf-3.1 @ 13717 on Tue Feb 28 18:10:10 2023 1677604210.0
frf-3.2 @ 13717 on Tue Feb 28 15:56:59 2023 1677596219.0
frf-4.1 @ 13717 on Tue Feb 28 18:17:17 2023 1677604637.0
frf-4.2 @ 13717 on Tue Feb 28 15:56:57 2023 1677596217.0
frf-5.1 @ 13717 on Tue Feb 28 18:20:00 2023 1677604800.0
frf-5.2 @ 13717 on Tue Feb 28 15:45:16 2023 1677595516.0
frf-6.1 @ 13717 on Tue Feb 28 18:46:40 2023 1677606400.0
frf-6.2 @ 13717 on Tue Feb 28 16:17:42 2023 1677597462.0
frf-7.1 @ 13717 on Tue Feb 28 18:16:29 2023 1677604589.0
frf-7.2 @ 13717 on Tue Feb 28 15:51:53 2023 1677595913.0
frf-8.1 @ 13717 on Tue Feb 28 18:24:21 2023 1677605061.0
frf-8.2 @ 13717 on Tue Feb 28 1

In [67]:
ret_dir = 'frf-585.0'
fname = os.path.join(result_dir, 'estimations', f'{ret_dir}/polyuq_imp.npz')
os.stat(fname).st_mtime > 1675778682.0

True

In [ ]:
ret_name = 'frf'
for freq_ind in range(1025):
    ret_ind = {'frequencies':freq_ind, 'space':0}
    ret_dir = f'{ret_name}-{".".join(str(e) for e in ret_ind.values())}'
    fname = os.path.join(result_dir, 'estimations', f'{ret_dir}/polyuq_imp.npz')
    
    if os.path.exists(fname):
        if os.stat(fname).st_mtime < 1675932359.0:
            os.rename(fname, fname + '.bak')
        else:
            
            print(time.ctime(os.stat(fname).st_mtime))
    else:
        print(ret_dir)
        f = open(fname,'wb')
        f.close()

In [ ]:
import shutil
N_mcs_ale = 13717 # N_mcs = 1e6 = N_mcs_ale * N_mcs_epi
N_mcs_epi = 729 # = 3^6 = 2.56^n_imp ~ 3^n_imp -> cover every corner and midpoints in a full-factorial design (but distributed)
result_dir = os.path.join('/usr/scratch4/sima9999/work/modal_uq/uq_modal_beam/', 'samples')
ale_dirs = []

In [ ]:
for jid in in_ds.ids:
    jid = jid.data.item()
    id_ale, id_epi = jid.split('_')
    
    ale_dir = os.path.join(result_dir,str(id_ale))
    if not ale_dir in ale_dirs:
        print(f'creating {ale_dir} total {len(ale_dirs)}')
        assert not os.path.exists(ale_dir)
        os.makedirs(ale_dir)
        assert os.path.exists(ale_dir)
        ale_dirs.append(ale_dir)
    elif ale_dir!=ale_dirs[-1]:
        continue
        
    srcfolder = os.path.join(result_dir, jid)
    dstfolder = os.path.join(ale_dir, str(id_epi))
    try:
        shutil.move(srcfolder,dstfolder)
    except Exception as e:
        print(e)
    

In [77]:
a = np.random.random((7,2))
a[2,:]=np.nan
# a = np.ma.array(a)
# a.mask = np.isnan(a)
ind = np.argsort(a, axis=0)
mask = np.any(~np.take_along_axis(np.isnan(a), ind, axis=0),axis=1)
display(mask)
ind = ind[mask,:]
# display(np.where(np.isnan(a)))
# a = a.data
display(a)
display(ind)
display(a[ind[:,0],0])
display(a[ind[:,1],1])

array([ True,  True,  True,  True,  True,  True, False])

array([[0.83476489, 0.00794258],
       [0.48414486, 0.98066388],
       [       nan,        nan],
       [0.23905244, 0.95371468],
       [0.13560081, 0.04894681],
       [0.59704103, 0.64009073],
       [0.85483053, 0.41306452]])

array([[4, 0],
       [3, 4],
       [1, 6],
       [5, 5],
       [0, 3],
       [6, 1]])

array([0.13560081, 0.23905244, 0.48414486, 0.59704103, 0.83476489,
       0.85483053])

array([0.00794258, 0.04894681, 0.41306452, 0.64009073, 0.95371468,
       0.98066388])